In [1]:
import json
import bz2
import sys
import os
import pandas as pd
import time
import matplotlib.pyplot as plt
import numpy as np

In [3]:
#specification of dataset reading parameters and folders path 
CHUNKSIZE=200
QUOTEBANK_POLITICIANS_USA=  'quotebank_politicians_USA.json.bz2' # US politician quotes
WIKI_DATA_FILTERED_LABELED = 'filtered_politician_labeled_v2.json.bz2' # filtered wikidata with interpretations
POLITICIANS_MENTIONS_USA= 'politicians_mentions_USA.json.bz2'#
qids_df=pd.read_json(WIKI_DATA_FILTERED_LABELED, lines=True)

In [4]:
politician_df=pd.read_json(WIKI_DATA_FILTERED_LABELED, lines=True)
politician_df.head()

,qid,name,gender,nationality,aliases,parties,positions held,religion,us_congress_id,candidacy_election
0,Q207,George W. Bush,male,Q30,"[George Walker Bush, Bush Jr., Dubya, GWB, Bus...",[Republican Party],"[Q5589725, [[+1995-01-17T00:00:00Z]], Q11696, ...",Q329646,,"[2000 United States presidential election, 200..."
1,Q946,Donald Tusk,male,Q36,[Donald Franciszek Tusk],"[Civic Platform, European People's Party]","[Q3259469, [[+2007-11-16T00:00:00Z]], Q735587,...",Q9592,,[2005 Polish presidential election]
2,Q1058,Narendra Modi,male,Q668,"[Modi, Narendra Bhai, Narendra Damodardas Modi...",[Bharatiya Janata Party],"[Q22337580, [[+2001-10-07T00:00:00Z]], Q425092...",Q9089,,[2014 Indian general election in Vadodara Lok ...
3,Q1253,Ban Ki-moon,male,Q884,"[Ban Kimoon, Ban Ki Moon]",[independent politician],"[Q81066, [[+2007-01-01T00:00:00Z]], [Foreign m...",Q9581,,[]
4,Q3996,V. P. Kalairajan,male,Q668,[],[All India Anna Dravida Munnetra Kazhagam],"[Q24558764, [[]]]",,,[]


In [5]:
politician_us_df = qids_df[qids_df['nationality']=='Q30']
politician_us_df.head()

,qid,name,gender,nationality,aliases,parties,positions held,religion,us_congress_id,candidacy_election
0,Q207,George W. Bush,male,Q30,"[George Walker Bush, Bush Jr., Dubya, GWB, Bus...",[Republican Party],"[Q5589725, [[+1995-01-17T00:00:00Z]], Q11696, ...",Q329646,,"[2000 United States presidential election, 200..."
5,Q4496,Mitt Romney,male,Q30,"[Willard Mitt Romney, Pierre Delecto]",[Republican Party],"[Q887117, [[+2003-01-02T00:00:00Z]], [bishop, ...",Q42504,R000615,"[2012 Republican Party presidential primaries,..."
7,Q11124,Stephen Breyer,male,Q30,[Stephen Gerald Breyer],[Democratic Party],"[Q11144, [[+1994-08-03T00:00:00Z]], Q23930295,...",Q9268,,[]
9,Q11674,David Paterson,male,Q30,[David Alexander Paterson],[Democratic Party],"[Q2347975, [[+2008-03-17T00:00:00Z]], Q6544428...",,,[]
23,Q22201,Jim DeMint,male,Q30,"[James Warren ""Jim"" DeMint, James Warren DeMint]",[Republican Party],"[[United States representative, []], [United S...",Q178169,D000595,[]


In [6]:
names_us=politician_us_df[['qid','name','aliases']]
names_us.head()

,qid,name,aliases
0,Q207,George W. Bush,"[George Walker Bush, Bush Jr., Dubya, GWB, Bus..."
5,Q4496,Mitt Romney,"[Willard Mitt Romney, Pierre Delecto]"
7,Q11124,Stephen Breyer,[Stephen Gerald Breyer]
9,Q11674,David Paterson,[David Alexander Paterson]
23,Q22201,Jim DeMint,"[James Warren ""Jim"" DeMint, James Warren DeMint]"


In [20]:
#extraction of political name's mention from the quotes
try:
    os.remove(POLITICIANS_MENTIONS_USA)
except:
    pass
total_time,total_mentions,chunk_number,nb_quote=0,0,0,0
mentions=[]
with bz2.open(POLITICIANS_MENTIONS_USA, 'wb') as bzout_mention_usa:
    with pd.read_json(QUOTEBANK_POLITICIANS_USA, lines=True, chunksize=CHUNKSIZE, compression='bz2' ) as df_reader:
        for chunk in df_reader:
            t1=time.time()
            for quote in chunk.quotation :
                
                for index,politic in names_us.iterrows():
                    if(politic['name'] in quote):
                        try:
                            mentions.append(dict(zip(['qid_quote', 'qid_speaker','qid_mentioned'], [chunk['quoteID'][nb_quote],chunk['qids'][nb_quote],politic['qid']])))# dump quoteID, and qid of speaker and cited
                            total_mentions+=1
                        except KeyError as e:
                            print(e,politic['qid'])
                    else :
                        for alias in politic['aliases']:
                            if(alias in quote):
                                try:
                                    mentions.append(dict(zip(['qid_quote', 'qid_speaker','qid_mentioned'], [chunk['quoteID'][nb_quote],chunk['qids'][nb_quote],politic['qid']])))#  dump quoteID, and qid of speaker and cited
                                    total_mentions+=1
                                except KeyError as e:
                                    print(e,politic['qid'])
                                break;
                nb_quote+=1
            t2=time.time()
            dt=t2-t1
            total_time+=dt
            chunk_number += 1
            print(chunk_number)
            print("from {} US political quotations there is {} mentions [time: {:.2f} s]".format(chunk_number*CHUNKSIZE, total_mentions, total_time))
            #bzout_mention_usa.write(('\n'.join(map(json.dumps, mentions))+'\n').encode('utf-8'))
            #json.dump(bzout_mention_usa,orient='records',lines=True)
            pd.DataFrame(mentions).to_json(path_or_buf=bzout_mention_usa,orient='records',lines=True)
            mentions = []     




1
from 200 US political quotations there is 11 mentions [time: 237.16]
2
from 400 US political quotations there is 39 mentions [time: 472.25]


KeyboardInterrupt: 

In [42]:
with pd.read_json(POLITICIANS_MENTIONS_USA, lines=True, chunksize=10000, compression='bz2' ) as df_reader:
    nb=0
    for chunk in df_reader:
        print(chunk)
            

            qid_quote  qid_speaker qid_mentioned
0   2020-02-07-012379     [Q22686]        Q22686
1   2020-01-29-019304   [Q3957020]         Q6279
2   2020-01-29-019304   [Q3957020]       Q311269
3   2020-01-02-019876    [Q185480]        Q22686
4   2020-01-27-036296      [Q6279]         Q6279
5   2020-04-09-027891      [Q1124]         Q1124
6   2020-01-10-048465      [Q6279]         Q6279
7   2020-01-23-056683    [Q359442]         Q6279
8   2020-01-25-036997  [Q18209402]        Q22686
9   2020-02-03-087410    [Q359442]        Q22686
10  2020-01-16-004972     [Q59315]        Q22686
11  2020-02-07-012791  [Q18978140]      Q7793121
12  2020-02-20-010482     [Q22686]       Q461391
13  2020-02-20-010482     [Q22686]       Q170581
14  2020-01-29-035877      [Q4496]         Q4496
15  2020-03-12-028343   [Q1281204]         Q6279
16  2020-01-29-045741      [Q4496]       Q311269
17  2020-01-12-020460    [Q923242]       Q473239
18  2020-01-12-020460    [Q923242]       Q311269
19  2020-03-03-03131

In [41]:
with pd.read_json(QUOTEBANK_POLITICIANS_USA, lines=True, chunksize=100000, compression='bz2' ) as df_reader:
    
    for chunk in df_reader:
        print(chunk) 
        #for q_quote,qid,quote in chunk:
        #print(q_quote)
    
            




                 quoteID                                          quotation  \
0      2020-01-16-000088  [ Department of Homeland Security ] was livid ...   
1      2020-03-19-000276  [ These ] actions will allow households who ha...   
2      2020-01-26-000499  a few of the candidates who will do better in ...   
3      2020-01-20-000982                        a host of other protections   
4      2020-01-03-001237  A lot of Americans [ were ] upset that this wa...   
...                  ...                                                ...   
99995  2020-03-22-043834  Where were the tests when we needed them? Wher...   
99996  2020-01-25-058535  Whether it's OK for the president to sacrifice...   
99997  2020-04-03-080197  While our hope is to return to classrooms befo...   
99998  2020-02-28-089830  While they were working on impeachment, we wer...   
99999  2020-02-01-061939  while this situation poses a very serious glob...   

                      speaker         qids         

KeyboardInterrupt: 

In [39]:
with pd.read_json(WIKI_DATA_FILTERED_LABELED, lines=True, chunksize=100000, compression='bz2' ) as df_reader:
    for chunk in df_reader:
        print(chunk)




              qid                  name gender nationality  \
0            Q207        George W. Bush   male         Q30   
1            Q946           Donald Tusk   male         Q36   
2           Q1058         Narendra Modi   male        Q668   
3           Q1253           Ban Ki-moon   male        Q884   
4           Q3996      V. P. Kalairajan   male        Q668   
...           ...                   ...    ...         ...   
99995  Q105840175  Frederick H. Magison   male         Q30   
99996  Q105840242  James Goodridge Page   male         Q30   
99997  Q105840296   William M. Robinson   male         Q30   
99998  Q105840297         Silas B. Root   male         Q30   
99999  Q105845441         Andres Taimla   male        Q191   

                                                 aliases  \
0      [George Walker Bush, Bush Jr., Dubya, GWB, Bus...   
1                               [Donald Franciszek Tusk]   
2      [Modi, Narendra Bhai, Narendra Damodardas Modi...   
3              